In [11]:
import pyarrow as pa
import pandas as pd
df = pd.DataFrame({'n_legs': [2, 4, 5, 100],
                   'animals': ["Flamingo", "Horse", "Brittle stars", "Centipede"]})
table = pa.Table.from_pandas(df)
print(table.to_batches()[0])


recBatchOutSchema = table.schema
recBatchOutColumns = table.column_names
recBatchOutDict = {}
for name in recBatchOutColumns:
    recBatchOutDict[name] = []
emptyRecBatchOut = pa.RecordBatch.from_pydict(
    mapping=recBatchOutDict, schema=recBatchOutSchema)

print(emptyRecBatchOut.column(0))

pyarrow.RecordBatch
n_legs: int64
animals: string
[]


In [3]:
import pyarrow as pa

# 定义两个RecordBatch对象
# 假设我们有两个包含相同列的RecordBatch

# 第一个RecordBatch的数据
data1 = [
    pa.array([1, 2, 3]),
    pa.array(['a', 'b', 'c'])
]

# 第一个RecordBatch
record_batch1 = pa.RecordBatch.from_arrays(data1, names=['id', 'letter'])

# 第二个RecordBatch的数据
data2 = [
    pa.array([4, 5, 6]),
    pa.array(['d', 'e', 'f'])
]

# 第二个RecordBatch
record_batch2 = pa.RecordBatch.from_arrays(data2, names=['id', 'letter'])

# 假设您有一个RecordBatch列表
record_batch_list = [record_batch1, record_batch2]

# 将所有RecordBatch合并成一个Table
table = pa.Table.from_batches(record_batch_list)

# 将Table转换回一个单一的RecordBatch
# 注意：这个操作假设所有RecordBatch的总大小可以放入内存
combined_record_batch = pa.RecordBatch.from_pandas(table.to_pandas())

# 现在 combined_record_batch 是一个包含所有数据的单一RecordBatch


In [8]:
from tqdm import tqdm
import time

# 例如，一个简单的循环
for i in tqdm(range(100),desc="test",total=10):
    # 模拟你的任务
    time.sleep(0.1)  # 模拟任务需要的时间


test: 100it [00:10,  9.56it/s]                       


In [26]:
import pyarrow as pa
import numpy as np
import pyarrow.compute as pc

array1 = pa.array(np.arange(1, 100_000_000 + 1, dtype=np.int64))
array2 = pa.array([2] * 100_000_000, type=pa.int64())

table = pa.RecordBatch.from_arrays([array1, array2], names=['a', 'b'])

def test():
    column_name = 'a'
    n = 3  # 分割成3个RecordBatch

    # 获取列值
    column_values = table.column(column_name)

    # 计算哈希值并进行模运算
    hash_mod_n = np.array([hash(value.as_py()) % n for value in column_values])

    # 初始化一个列表来存储分割后的 RecordBatches
    record_batches = [table.filter(pa.array(hash_mod_n == i)) for i in range(n)]


from memory_profiler import memory_usage

mem_usage = memory_usage(test)

print(mem_usage)


/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/resource_tracker.py:104: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '


[460.234375, 460.859375, 470.140625, 479.3125, 482.9375, 492.515625, 502.140625, 511.9375, 521.75, 531.53125, 541.296875, 550.984375, 560.671875, 570.140625, 579.546875, 589.28125, 598.96875, 608.734375, 618.5625, 628.3125, 638.109375, 647.859375, 657.703125, 667.46875, 677.15625, 687.015625, 696.828125, 598.515625, 608.3125, 618.125, 627.953125, 637.78125, 647.625, 657.53125, 667.21875, 676.84375, 686.65625, 696.40625, 706.1875, 715.984375, 725.796875, 735.546875, 745.328125, 755.125, 764.921875, 774.71875, 784.484375, 794.359375, 803.859375, 813.65625, 823.5, 833.3125, 843.03125, 852.5625, 862.34375, 872.296875, 882.109375, 891.984375, 901.828125, 911.609375, 921.390625, 931.125, 940.96875, 950.8125, 960.59375, 970.390625, 980.28125, 990.0625, 999.859375, 1009.625, 1019.46875, 1029.21875, 1038.90625, 1048.515625, 1058.296875, 1068.046875, 1077.78125, 1087.46875, 1097.25, 1067.984375, 1077.734375, 1087.515625, 1097.296875, 1107.078125, 1116.796875, 1126.546875, 1088.671875, 1098.4375,